## Prepare challenge dataset

In this notebook we are taking the chatgpt generated examples (and the special case manual examples). And preparing in a json file.

 Organized hierarchically under "capabilities" that represent distinct linguistic phenomena (distance, spatial/temporal distinctions, dative alternations, negation, head noun identification, and predicate disambiguation), the dataset employs four testing formats: MFT (Minimal Functional Test) for single-sentence evaluations, DIR (Directional) for paired sentences with different expected labels, INV (Invariant) for paired sentences with consistent labeling, and specialDIR for specialized predicate positioning tests. Each example contains tokenized sentences with specified predicate and argument indices along with their expected semantic role labels (like ARG0, ARG2, ARGM-TMP, ARGM-LOC), allowing for systematic assessment of a model's understanding of semantic roles across syntactic variations and linguistic challenges.

In [178]:
import pandas as pd
import spacy
import numpy as np
import json

How we store tests.

```
MFT:(tokenized, predicate_index, target_index, target_SRL) 
DIR:(tokenized1,tokenized2,predicate_index,target_index1, target_index2, target_SRL1, target_SRL2), 
INV: (tokenized1,tokenized2,predicate_index,target_index1,target_index2, target_SRL)
specialDIR: (tokenized1, tokenized2, predicate_index1, predicate_index2, SRL)
```

In [179]:
dataset = {
    "version": "1.0",
    "description": "Semantic Role Labeling Challenge Dataset"
}

In [180]:
def add_examples(dataset, capability, test_type, test_name, *args):
    """
    Adds examples to the dataset directly from lists of data.
    
    Parameters:
    - dataset: The dataset dictionary to add examples to
    - capability: String identifying the capability (e.g., 'distance', 'spacetemp')
    - test_type: String indicating the format ('MFT', 'DIR', 'INV', 'specialDIR')
    - test_name: String providing a unique name for this specific test
    - *args: Lists of data appropriate for the test type (see function body for details)
    
    Returns:
    - The updated dataset dictionary
    """
    if "capabilities" not in dataset:
        dataset["capabilities"] = {}
    
    if capability not in dataset["capabilities"]:
        dataset["capabilities"][capability] = {}
    
    if test_type not in dataset["capabilities"][capability]:
        dataset["capabilities"][capability][test_type] = {}
    
    if test_name not in dataset["capabilities"][capability][test_type]:
        dataset["capabilities"][capability][test_type][test_name] = []
    
    examples = []
    
    if test_type == "MFT":
        # For MFT: add_examples(dataset, "capability", "MFT", tokenized_list, predicate_indexes, target_indexes, target_srls)
        if len(args) != 4:
            raise ValueError("MFT format requires 4 lists: tokenized, predicate_indexes, target_indexes, target_srls")
        
        tokenized_list, predicate_indexes, target_indexes, target_srls = args
        
        for i in range(len(tokenized_list)):
            example = {
                "tokenized": tokenized_list[i],
                "predicate_index": predicate_indexes[i],
                "target_index": target_indexes[i],
                "target_SRL": target_srls[i]
            }
            examples.append(example)
    
    elif test_type == "DIR":
        # For DIR: add_examples(dataset, "capability", "DIR", tokenized_list1, tokenized_list2, 
        #                      predicate_indexes, target_indexes1, target_indexes2, target_srls1, target_srls2)
        if len(args) != 7:
            raise ValueError("DIR format requires 7 lists: tokenized1, tokenized2, predicate_indexes, " +
                           "target_indexes1, target_indexes2, target_srls1, target_srls2")
        
        tokenized_list1, tokenized_list2, predicate_indexes, target_indexes1, target_indexes2, target_srls1, target_srls2 = args
        
        for i in range(min(len(tokenized_list1), len(tokenized_list2))):
            example = {
                "tokenized1": tokenized_list1[i],
                "tokenized2": tokenized_list2[i],
                "predicate_index": predicate_indexes[i],
                "target_index1": target_indexes1[i],
                "target_index2": target_indexes2[i],
                "target_SRL1": target_srls1[i],
                "target_SRL2": target_srls2[i]
            }
            examples.append(example)
    
    elif test_type == "INV":
        # For INV: add_examples(dataset, "capability", "INV", tokenized_list1, tokenized_list2, 
        #                      predicate_indexes, target_indexes1, target_indexes2, target_srls)
        if len(args) != 6:
            raise ValueError("INV format requires 6 lists: tokenized1, tokenized2, predicate_indexes, " +
                           "target_indexes1, target_indexes2, target_srls")
        
        tokenized_list1, tokenized_list2, predicate_indexes, target_indexes1, target_indexes2, target_srls = args
        
        for i in range(min(len(tokenized_list1), len(tokenized_list2))):
            example = {
                "tokenized1": tokenized_list1[i],
                "tokenized2": tokenized_list2[i],
                "predicate_index": predicate_indexes[i],
                "target_index1": target_indexes1[i],
                "target_index2": target_indexes2[i],
                "target_SRL": target_srls[i]
            }
            examples.append(example)
    
    elif test_type == "specialDIR":
        # For specialDIR: add_examples(dataset, "capability", "specialDIR", tokenized_list1, tokenized_list2, 
        #                             predicate_indexes1, predicate_indexes2, srls)
        if len(args) != 5:
            raise ValueError("specialDIR format requires 5 lists: tokenized1, tokenized2, " +
                           "predicate_indexes1, predicate_indexes2, srls")
        
        tokenized_list1, tokenized_list2, predicate_indexes1, predicate_indexes2, srls = args
        
        for i in range(min(len(tokenized_list1), len(tokenized_list2))):
            example = {
                "tokenized1": tokenized_list1[i],
                "tokenized2": tokenized_list2[i],
                "predicate_index1": predicate_indexes1[i],
                "predicate_index2": predicate_indexes2[i],
                "SRL": srls[i]
            }
            examples.append(example)
    
    else:
        raise ValueError(f"Unknown test type: {test_type}")
    
    dataset["capabilities"][capability][test_type][test_name].extend(examples)
    
    print(f"Added {len(examples)} examples for capability '{capability}' with test type '{test_type}' and name '{test_name}'")
    return dataset

### Distance

MFT: Test assignment of ARG0 when a parenthetical phrase separates the subject and predicate.

In [181]:
df = pd.read_csv('distance1.csv')
nlp = spacy.load('en_core_web_sm')

def adjust_indices(sentence, original_indices):
    '''
    This function adjusts word indices from a simple space-based tokenization to match the indices of tokens
    generated by the spaCy tokenizer. It takes a sentence and a list of original indices as input. The function
    first tokenizes the sentence using both space-based and spaCy tokenization methods. It then maps the original
    indices to spaCy token indices by finding the best character position overlap between the two tokenizations.
    This is necessary because spaCy's tokenization can differ from simple space-based tokenization, affecting
    the alignment of indices. The function returns a list of adjusted indices that correspond to the spaCy tokens.
    '''
    original_tokens = sentence.split()
    doc = nlp(sentence)
    
    original_char_spans = []
    current_pos = 0
    for token in original_tokens:
        token_pos = sentence[current_pos:].find(token) + current_pos
        original_char_spans.append((token_pos, token_pos + len(token)))
        current_pos = token_pos + len(token)
    
    adjusted_indices = []
    for idx in original_indices:
        if 0 <= idx < len(original_char_spans):
            orig_start, orig_end = original_char_spans[idx]
            
            best_token_idx = None
            best_overlap = -1
            
            for i, token in enumerate(doc):
                token_start = token.idx
                token_end = token_start + len(token.text)
                
                overlap_start = max(orig_start, token_start)
                overlap_end = min(orig_end, token_end)
                overlap = max(0, overlap_end - overlap_start)
                
                if overlap > best_overlap:
                    best_overlap = overlap
                    best_token_idx = i
            
            if best_token_idx is not None:
                adjusted_indices.append(best_token_idx)
            else:
                adjusted_indices.append(idx)
        else:
            adjusted_indices.append(idx)
            
    return adjusted_indices

In [182]:
adjusted_data = []
for _, row in df.iterrows():
    sentence = row['Sentence']
    arg0_index = row['ARG0_Index']
    pred_index = row['Predicate_Index']
    
    # Adjust indices
    adjusted_indices = adjust_indices(sentence, [arg0_index, pred_index])
    
    adjusted_data.append({
        'Sentence': sentence,
        'Original_ARG0_index': arg0_index,
        'Original_Predicate_index': pred_index,
        'Adjusted_ARG0_index': adjusted_indices[0],
        'Adjusted_Predicate_index': adjusted_indices[1],
        'SpaCy_Tokens': [token.text for token in nlp(sentence)]
    })

# Create new DataFrame with adjusted indices
distance1 = pd.DataFrame(adjusted_data)

# Display the results
distance1.head()

,Sentence,Original_ARG0_index,Original_Predicate_index,Adjusted_ARG0_index,Adjusted_Predicate_index,SpaCy_Tokens
0,"Jack, despite having an extremely busy schedul...",0,10,0,12,"[Jack, ,, despite, having, an, extremely, busy..."
1,"Mary, although feeling under the weather, prep...",0,6,0,8,"[Mary, ,, although, feeling, under, the, weath..."
2,"My neighbor, who recently moved in, mowed the ...",1,6,1,8,"[My, neighbor, ,, who, recently, moved, in, ,,..."
3,"The teacher, after grading a mountain of paper...",1,8,1,10,"[The, teacher, ,, after, grading, a, mountain,..."
4,"Bob, having finished his homework early, watch...",0,6,0,8,"[Bob, ,, having, finished, his, homework, earl..."


In [183]:
# Call the first row column SpaCy_tokens
first_row_spacy_tokens = distance1.iloc[85]['SpaCy_Tokens']
first_row_spacy_tokens[0]


'Hailey'

In [184]:
tokenized_list = [row["SpaCy_Tokens"] for _, row in distance1.iterrows()]
predicate_indexes = [row["Adjusted_Predicate_index"] for _, row in distance1.iterrows()]
target_indexes = [row["Adjusted_ARG0_index"] for _, row in distance1.iterrows()]
target_srls = ["ARG0"] * len(tokenized_list)  

dataset = add_examples(
    dataset,
    "distance",
    "MFT",
    "1",
    tokenized_list,
    predicate_indexes,
    target_indexes,
    target_srls
)

Added 86 examples for capability 'distance' with test type 'MFT' and name '1'


MFT: Test consistent ARG0 assignment across multiple predicates.

In [185]:
df = pd.read_csv('distance2.csv')

adjusted_data = []
for _, row in df.iterrows():
    sentence = row['Sentence']
    arg0_index = row['ARG0_Index']
    pred_index = row['Last_Predicate_Index']
    
    adjusted_indices = adjust_indices(sentence, [arg0_index, pred_index])
    
    adjusted_data.append({
        'Sentence': sentence,
        'Original_ARG0_index': arg0_index,
        'Original_Predicate_index': pred_index,
        'Adjusted_ARG0_index': adjusted_indices[0],
        'Adjusted_Predicate_index': adjusted_indices[1],
        'SpaCy_Tokens': [token.text for token in nlp(sentence)]
    })

distance2 = pd.DataFrame(adjusted_data)

distance2.head()

,Sentence,Original_ARG0_index,Original_Predicate_index,Adjusted_ARG0_index,Adjusted_Predicate_index,SpaCy_Tokens
0,John kicked the ball and scored a goal.,0,5,0,5,"[John, kicked, the, ball, and, scored, a, goal..."
1,Alice painted the canvas and sold the painting.,0,5,0,5,"[Alice, painted, the, canvas, and, sold, the, ..."
2,Robert cooked dinner and cleaned the kitchen.,0,4,0,4,"[Robert, cooked, dinner, and, cleaned, the, ki..."
3,Mary read a book and wrote a review.,0,5,0,5,"[Mary, read, a, book, and, wrote, a, review, .]"
4,Sam drove a car and parked it.,0,5,0,5,"[Sam, drove, a, car, and, parked, it, .]"


In [186]:
# Call the first row column SpaCy_tokens
first_row_spacy_tokens = distance2.iloc[97]['SpaCy_Tokens']
first_row_spacy_tokens[5] 


'restored'

In [187]:
tokenized_list = [row["SpaCy_Tokens"] for _, row in distance2.iterrows()]
predicate_indexes = [row["Adjusted_Predicate_index"] for _, row in distance2.iterrows()]
target_indexes = [row["Adjusted_ARG0_index"] for _, row in distance2.iterrows()]
target_srls = ["ARG0"] * len(tokenized_list)  

dataset = add_examples(
    dataset,
    "distance",
    "MFT",
    "2",
    tokenized_list,
    predicate_indexes,
    target_indexes,
    target_srls
)

Added 100 examples for capability 'distance' with test type 'MFT' and name '2'


### SpaceTemp

DIR: Test that switching a modifier from temporal to spatial yields different adjunct roles.

In [188]:
df = pd.read_csv('spacetemp.csv')

adjusted_data = []
for _, row in df.iterrows():
    sentence = row['SentenceTemp']
    argtemp_index = row['ARGM-TMP_Index']
    pred_index = row['Predicate_Index']
    
    # Adjust indices
    adjusted_indices = adjust_indices(sentence, [argtemp_index, pred_index])
    
    adjusted_data.append({
        'SentenceTemp': sentence,
        'Original_ARGM-TMP_index': argtemp_index,
        'Original_Predicate_index': pred_index,
        'Adjusted_ARGM-TMP_index': adjusted_indices[0],
        'Adjusted_Predicate_index': adjusted_indices[1],
        'SpaCy_Tokens': [token.text for token in nlp(sentence)]
    })

# Create new DataFrame with adjusted indices
temp = pd.DataFrame(adjusted_data)

# Display the results
temp.head()

,SentenceTemp,Original_ARGM-TMP_index,Original_Predicate_index,Adjusted_ARGM-TMP_index,Adjusted_Predicate_index,SpaCy_Tokens
0,John met Jack on Friday.,4,1,4,1,"[John, met, Jack, on, Friday, .]"
1,Mark delivered the package at 3 PM.,6,1,6,1,"[Mark, delivered, the, package, at, 3, PM, .]"
2,Emma started the presentation at dawn.,5,1,5,1,"[Emma, started, the, presentation, at, dawn, .]"
3,Lucas finished the race at sunset.,5,1,5,1,"[Lucas, finished, the, race, at, sunset, .]"
4,Olivia began the meeting at 10 AM.,6,1,6,1,"[Olivia, began, the, meeting, at, 10, AM, .]"


In [189]:
# Call the first row column SpaCy_tokens
first_row_spacy_tokens = temp.iloc[50]['SpaCy_Tokens']
first_row_spacy_tokens[6] 

'AM'

In [190]:
tokenized_list1 = [row["SpaCy_Tokens"] for _, row in  temp.iterrows()]
predicate_indexes = [row["Adjusted_Predicate_index"] for _, row in temp.iterrows()]
target_index1 = [row["Adjusted_ARGM-TMP_index"] for _, row in temp.iterrows()]
target_srls1 = ["ARGM-TMP"] * len(tokenized_list)  

In [191]:
df = pd.read_csv('spacetemp.csv')

adjusted_data = []
for _, row in df.iterrows():
    sentence = row['SentenceLoc']
    argloc_index = row['ARGM-LOC_Index']
    pred_index = row['Predicate_Index']
    
    # Adjust indices
    adjusted_indices = adjust_indices(sentence, [argloc_index, pred_index])
    
    adjusted_data.append({
        'SentenceLoc': sentence,
        'Original_ARGM-LOC_index': argloc_index,
        'Original_Predicate_index': pred_index,
        'Adjusted_ARGM-LOC_index': adjusted_indices[0],
        'Adjusted_Predicate_index': adjusted_indices[1],
        'SpaCy_Tokens': [token.text for token in nlp(sentence)]
    })

# Create new DataFrame with adjusted indices
space = pd.DataFrame(adjusted_data)

# Display the results
space.head()

,SentenceLoc,Original_ARGM-LOC_index,Original_Predicate_index,Adjusted_ARGM-LOC_index,Adjusted_Predicate_index,SpaCy_Tokens
0,John met Jack under a bridge.,5,1,5,1,"[John, met, Jack, under, a, bridge, .]"
1,Mark delivered the package at the post office.,7,1,7,1,"[Mark, delivered, the, package, at, the, post,..."
2,Emma started the presentation near the auditor...,6,1,6,1,"[Emma, started, the, presentation, near, the, ..."
3,Lucas finished the race on the track.,6,1,6,1,"[Lucas, finished, the, race, on, the, track, .]"
4,Olivia began the meeting at the conference room.,7,1,7,1,"[Olivia, began, the, meeting, at, the, confere..."


In [192]:
tokenized_list2 = [row["SpaCy_Tokens"] for _, row in space .iterrows()]
target_index2 = [row["Adjusted_ARGM-LOC_index"] for _, row in space.iterrows()]
target_srls2 = ["ARGM-LOC"] * len(tokenized_list)  

In [193]:
dataset = add_examples(
    dataset,
    "spacetemp",
    "DIR",
    "1",
    tokenized_list1,
    tokenized_list2,
    predicate_indexes,
    target_index1,
    target_index2,
    target_srls1,
    target_srls2
)

Added 59 examples for capability 'spacetemp' with test type 'DIR' and name '1'


### DativeAlter

INV: Verify that active and prepositional constructions consistently label the recipient (indirect object) as ARG2 

In [194]:
df = pd.read_csv('dative.csv')

adjusted_data = []
for _, row in df.iterrows():
    sentence = row['SentenceActive']
    arg2_index = row['Active_ARG2_Index']
    pred_index = row['Predicate_Index']
    
    # Adjust indices
    adjusted_indices = adjust_indices(sentence, [arg2_index, pred_index])
    
    adjusted_data.append({
        'SentenceActive': sentence,
        'Original_ARG2_index': arg2_index,
        'Original_Predicate_index': pred_index,
        'Adjusted_ARG2_index': adjusted_indices[0],
        'Adjusted_Predicate_index': adjusted_indices[1],
        'SpaCy_Tokens': [token.text for token in nlp(sentence)]
    })

# Create new DataFrame with adjusted indices
dative_active = pd.DataFrame(adjusted_data)

# Display the results
dative_active.head()

,SentenceActive,Original_ARG2_index,Original_Predicate_index,Adjusted_ARG2_index,Adjusted_Predicate_index,SpaCy_Tokens
0,I gave John a book.,2,1,2,1,"[I, gave, John, a, book, .]"
1,She sent Mary a letter.,2,1,2,1,"[She, sent, Mary, a, letter, .]"
2,He offered Tom a ride.,2,1,2,1,"[He, offered, Tom, a, ride, .]"
3,They handed Mark a gift.,2,1,2,1,"[They, handed, Mark, a, gift, .]"
4,We provided Jane a chance.,2,1,2,1,"[We, provided, Jane, a, chance, .]"


In [195]:
tokenized_list1 = [row["SpaCy_Tokens"] for _, row in  dative_active.iterrows()]
predicate_indexes = [row["Adjusted_Predicate_index"] for _, row in dative_active.iterrows()]
target_index1 = [row["Adjusted_ARG2_index"] for _, row in dative_active.iterrows()]
target_srls = ["ARG2"] * len(tokenized_list)  

In [196]:
df = pd.read_csv('dative.csv')

adjusted_data = []
for _, row in df.iterrows():
    sentence = row['SentencePrepositional']
    arg2_index = row['Prepositional_ARG2_Index']
    pred_index = row['Predicate_Index']
    
    # Adjust indices
    adjusted_indices = adjust_indices(sentence, [arg2_index, pred_index])
    
    adjusted_data.append({
        'SentencePrepositional': sentence,
        'Original_ARG2_index': arg2_index,
        'Original_Predicate_index': pred_index,
        'Adjusted_ARG2_index': adjusted_indices[0],
        'Adjusted_Predicate_index': adjusted_indices[1],
        'SpaCy_Tokens': [token.text for token in nlp(sentence)]
    })

# Create new DataFrame with adjusted indices
dative_prep = pd.DataFrame(adjusted_data)

# Display the results
dative_prep.head()

,SentencePrepositional,Original_ARG2_index,Original_Predicate_index,Adjusted_ARG2_index,Adjusted_Predicate_index,SpaCy_Tokens
0,I gave a book to John.,5,1,5,1,"[I, gave, a, book, to, John, .]"
1,She sent a letter to Mary.,5,1,5,1,"[She, sent, a, letter, to, Mary, .]"
2,He offered a ride to Tom.,5,1,5,1,"[He, offered, a, ride, to, Tom, .]"
3,They handed a gift to Mark.,5,1,5,1,"[They, handed, a, gift, to, Mark, .]"
4,We provided a chance to Jane.,5,1,5,1,"[We, provided, a, chance, to, Jane, .]"


In [197]:
tokenized_list2 = [row["SpaCy_Tokens"] for _, row in  dative_prep.iterrows()]
target_index2 = [row["Adjusted_ARG2_index"] for _, row in dative_prep.iterrows()]

In [198]:
dataset = add_examples(
    dataset,
    "dativealter",
    "INV",
    "1",
    tokenized_list1,
    tokenized_list2,
    predicate_indexes,
    target_index1,
    target_index2,
    target_srls
)

Added 57 examples for capability 'dativealter' with test type 'INV' and name '1'


### Negation

In [199]:
df = pd.read_csv('negation.csv')

adjusted_data = []
for _, row in df.iterrows():
    sentence = row['Sentence']
    pred_index = row['Predicate_Index']
    neg_index = row['ARGM_NEG_Index']

    adjusted_indices = adjust_indices(sentence, [pred_index, neg_index])
    
    adjusted_data.append({
        'Sentence': sentence,
        'Original_Predicate_index': pred_index,
        'Original_ARGM_NEG_index': neg_index,
        'Adjusted_Predicate_index': adjusted_indices[0],
        'Adjusted_ARGM_NEG_index': adjusted_indices[1],
        'SpaCy_Tokens': [token.text for token in nlp(sentence)]
    })

negation = pd.DataFrame(adjusted_data)

negation.head()

,Sentence,Original_Predicate_index,Original_ARGM_NEG_index,Adjusted_Predicate_index,Adjusted_ARGM_NEG_index,SpaCy_Tokens
0,Jack did not clean the house.,3,2,3,2,"[Jack, did, not, clean, the, house, .]"
1,Maria does not like spicy food.,3,2,3,2,"[Maria, does, not, like, spicy, food, .]"
2,The manager can not approve the proposal.,4,3,4,3,"[The, manager, can, not, approve, the, proposa..."
3,He will not attend the meeting tomorrow.,3,2,3,2,"[He, will, not, attend, the, meeting, tomorrow..."
4,We can not find any evidence of the claim.,3,2,3,2,"[We, can, not, find, any, evidence, of, the, c..."


In [200]:
tokenized_list = [row["SpaCy_Tokens"] for _, row in negation.iterrows()]
predicate_indexes = [row["Adjusted_Predicate_index"] for _, row in negation.iterrows()]
target_indexes = [row["Adjusted_ARGM_NEG_index"] for _, row in negation.iterrows()]
target_srls = ["ARGM-NEG"] * len(tokenized_list)  

dataset = add_examples(
    dataset,
    "negation",
    "MFT",
    "1",
    tokenized_list,
    predicate_indexes,
    target_indexes,
    target_srls
)

Added 50 examples for capability 'negation' with test type 'MFT' and name '1'


### Head Extraction

MFT: Test that the model correctly identifies the head noun from a noun phrase with a single (participial) adjective.

In [201]:
df = pd.read_csv('head1.csv')

adjusted_data = []
for _, row in df.iterrows():
    sentence = row['Sentence']
    pred_index = row['Predicate_Index']
    arg0_index = row['ARG0_Index']

    adjusted_indices = adjust_indices(sentence, [pred_index, arg0_index])
    
    adjusted_data.append({
        'Sentence': sentence,
        'Original_Predicate_index': pred_index,
        'Original_ARG0_index': arg0_index,
        'Adjusted_Predicate_index': adjusted_indices[0],
        'Adjusted_ARG0_index': adjusted_indices[1],
        'SpaCy_Tokens': [token.text for token in nlp(sentence)]
    })

head1 = pd.DataFrame(adjusted_data)
head1.head()

,Sentence,Original_Predicate_index,Original_ARG0_index,Adjusted_Predicate_index,Adjusted_ARG0_index,SpaCy_Tokens
0,The red car zoomed down the highway.,3,2,3,2,"[The, red, car, zoomed, down, the, highway, .]"
1,Tall buildings collapsed suddenly.,2,1,2,1,"[Tall, buildings, collapsed, suddenly, .]"
2,The ancient tree fell in the storm.,3,2,3,2,"[The, ancient, tree, fell, in, the, storm, .]"
3,A broken vase shattered on the floor.,3,2,3,2,"[A, broken, vase, shattered, on, the, floor, .]"
4,Sleepy cats purred softly.,2,1,2,1,"[Sleepy, cats, purred, softly, .]"


In [202]:
tokenized_list = [row["SpaCy_Tokens"] for _, row in head1.iterrows()]
predicate_indexes = [row["Adjusted_Predicate_index"] for _, row in head1.iterrows()]
target_indexes = [row["Adjusted_ARG0_index"] for _, row in head1.iterrows()]
target_srls = ["ARG0"] * len(tokenized_list)  

dataset = add_examples(
    dataset,
    "head",
    "MFT",
    "1",
    tokenized_list,
    predicate_indexes,
    target_indexes,
    target_srls
)

Added 50 examples for capability 'head' with test type 'MFT' and name '1'


MFT: Test that the model correctly identifies the head noun from a noun phrase with multiple (participial) adjectives.

In [203]:
df = pd.read_csv('head2.csv')

adjusted_data = []
for _, row in df.iterrows():
    sentence = row['Sentence']
    pred_index = row['Predicate_Index']
    arg0_index = row['ARG0_Index']

    adjusted_indices = adjust_indices(sentence, [pred_index, arg0_index])
    
    adjusted_data.append({
        'Sentence': sentence,
        'Original_Predicate_index': pred_index,
        'Original_ARG0_index': arg0_index,
        'Adjusted_Predicate_index': adjusted_indices[0],
        'Adjusted_ARG0_index': adjusted_indices[1],
        'SpaCy_Tokens': [token.text for token in nlp(sentence)]
    })

head2 = pd.DataFrame(adjusted_data)
head2.head()

,Sentence,Original_Predicate_index,Original_ARG0_index,Adjusted_Predicate_index,Adjusted_ARG0_index,SpaCy_Tokens
0,"The old, abandoned building fell.",4,3,5,4,"[The, old, ,, abandoned, building, fell, .]"
1,"The bright, shining star twinkled in the night.",4,3,5,4,"[The, bright, ,, shining, star, twinkled, in, ..."
2,"The large, rumpled suitcase arrived at the sta...",4,3,5,4,"[The, large, ,, rumpled, suitcase, arrived, at..."
3,"The green, overgrown garden flourished in spring.",4,3,5,4,"[The, green, ,, overgrown, garden, flourished,..."
4,"The elegant, Victorian mansion stood silently ...",4,3,5,4,"[The, elegant, ,, Victorian, mansion, stood, s..."


In [204]:
tokenized_list = [row["SpaCy_Tokens"] for _, row in head2.iterrows()]
predicate_indexes = [row["Adjusted_Predicate_index"] for _, row in head2.iterrows()]
target_indexes = [row["Adjusted_ARG0_index"] for _, row in head2.iterrows()]
target_srls = ["ARG0"] * len(tokenized_list)  

dataset = add_examples(
    dataset,
    "head",
    "MFT",
    "2",
    tokenized_list,
    predicate_indexes,
    target_indexes,
    target_srls
)

Added 50 examples for capability 'head' with test type 'MFT' and name '2'


### Ambigous Predicate

Manually select predicate positions

In [205]:
without_arg=[
    "He assembled enemies into a united group.",
    "The dog heeled to me in obedience.",
    "John forwent to the circus.",
    "She picked at her pasta from the plate.",
    "John stresses about environmental awareness.",
    "We cut a truce with them for peace."
]

with_arg=[
    "John's assembly of the furniture from a kit to a home.",
    "He heeled the bike over.",
    "Benefits are forgone to the private sector.",
    "You can't pick grapes from thornbushes.",
    "He was stressed about the test.",
    "Google cut its price to five dollars."
]

SRL_oi=["ARG4", "ARG2", "ARG3", "ARG2", "ARG3", "ARG4"]

tokenized_without_arg = [[token.text for token in nlp(sentence)] for sentence in without_arg]
tokenized_with_arg = [[token.text for token in nlp(sentence)] for sentence in with_arg]


In [206]:
print(tokenized_without_arg)
print(tokenized_with_arg)

[['He', 'assembled', 'enemies', 'into', 'a', 'united', 'group', '.'], ['The', 'dog', 'heeled', 'to', 'me', 'in', 'obedience', '.'], ['John', 'forwent', 'to', 'the', 'circus', '.'], ['She', 'picked', 'at', 'her', 'pasta', 'from', 'the', 'plate', '.'], ['John', 'stresses', 'about', 'environmental', 'awareness', '.'], ['We', 'cut', 'a', 'truce', 'with', 'them', 'for', 'peace', '.']]
[['John', "'s", 'assembly', 'of', 'the', 'furniture', 'from', 'a', 'kit', 'to', 'a', 'home', '.'], ['He', 'heeled', 'the', 'bike', 'over', '.'], ['Benefits', 'are', 'forgone', 'to', 'the', 'private', 'sector', '.'], ['You', 'ca', "n't", 'pick', 'grapes', 'from', 'thornbushes', '.'], ['He', 'was', 'stressed', 'about', 'the', 'test', '.'], ['Google', 'cut', 'its', 'price', 'to', 'five', 'dollars', '.']]


In [207]:
predicate1=[1, 2, 1, 1, 1, 1]
predicate2=[2, 1, 2, 3, 2, 1]

In [208]:
dataset = add_examples(
    dataset,
    "predicatedis",
    "specialDIR",
    "1",
    tokenized_without_arg,
    tokenized_with_arg,
    predicate1,
    predicate2,
    SRL_oi
)

Added 6 examples for capability 'predicatedis' with test type 'specialDIR' and name '1'


In [209]:
with open('dataset.json', 'w') as json_file:
    json.dump(dataset, json_file, indent=4)